In [1]:
%run utils.ipynb
random.seed(1)

In [2]:
root_dir = "./files/4_split_400/"

In [3]:
class_dirs = [os.path.join(root_dir, i) for i in os.listdir(root_dir)]
random.shuffle(class_dirs)
print(f"{len(class_dirs)} classes")

400 classes


In [4]:
class_dirs = class_dirs[:300]

In [5]:
x_train, y_train = [], []
x_test, y_test = [], []

In [6]:
%%time
for class_label, class_dir in tqdm(enumerate(class_dirs)):
    train_list, test_list = list_split(get_all_images(class_dir, shuffle = False),
                                       ratio = [0.4, 0.6], shuffle = False)
    try:
        for image_path in train_list:    
            _, _, roi = roi_extraction_with_hood(image_path)
            roi = resize(roi, is_gray = True, width = 256, height = 128)
            feature = cvhog.compute(roi)
            x_train.append(feature)
            y_train.append(class_label)
 
        for image_path in test_list:    
            _, _, roi = roi_extraction_with_hood(image_path)
            roi = resize(roi, is_gray = True, width = 256, height = 128)
            feature = cvhog.compute(roi)
            x_test.append(feature)
            y_test.append(class_label)
    except:
        print(image_path)

files/4_split_400/广汽传祺--广汽传祺--传祺GS5--2012--2013--2014/传祺GS5__传祺GS5__2013款1.8T自动四驱尊贵版__9070813016674127f787cbfa4769270e1ff6fc18.jpg
files/4_split_400/吉利v3--吉利--金刚--2009/吉利金刚2代__金刚2代__2009款1.5L—MT标准型__cf5393ba1dc83d6bf64bd1b2f4588a5e27a4db43.jpg
files/4_split_400/奇瑞--奇瑞汽车--风云2--两厢--2010--三厢--2010/风云2两厢__风云2两厢__2010款1.5—MT进取型__5e375c6383324ac428eb8999a3cddd9bff57d200.jpg
files/4_split_400/菲亚特--广汽菲亚特--致悦--2014_v2--2015/致悦__致悦__2014款1.4T双离合豪华运动版__9447830e9d2d4142d9b1a1d046597cc1ed5042e6.jpg
files/4_split_400/别克--上海通用别克--凯越--2013/凯越__凯越__2013款1.5L手动尊享型__04cde12d91a26e1c3ebab78501ba49525638a7d0.jpg


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


files/4_split_400/奇瑞--奇瑞汽车--奇瑞QQ3--2004--2007--2010--2011--2012_v1/奇瑞QQ3__QQ3__2010款0.8手动冠军版__88b1680a13405d1a64b6a638aec0368fe2f85f2c.jpg

CPU times: user 7min 24s, sys: 9.82 s, total: 7min 34s
Wall time: 4min 42s


In [7]:
f_dim = x_test[0].shape[0]
print(f"feature dim : {f_dim}")

feature dim : 756


In [8]:
x_train = np.asarray(x_train).reshape(-1, f_dim)
x_test = np.asarray(x_test).reshape(-1, f_dim)

In [9]:
x_train.shape

(3818, 756)

In [10]:
%%time
# Grid Search
# Parameter Grid
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.00001, 10]}
 
# Make grid search classifier
clf_grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv = 3, n_jobs = -1) #, pre_dispatch = cpu_count//2
 
# Train the classifier
clf_grid.fit(x_train, y_train)

# clf = grid.best_estimator_()
print("Best Parameters:\n", clf_grid.best_params_)
print("Best Estimators:\n", clf_grid.best_estimator_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  3.3min finished


Best Parameters:
 {'C': 100, 'gamma': 0.01}
Best Estimators:
 SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
CPU times: user 14 s, sys: 59.8 ms, total: 14 s
Wall time: 3min 29s


In [11]:
# Make predictions on unseen test data
y_pred = clf_grid.predict(x_test)
# print("Accuracy: {}%".format(clf_grid.score(x_test, y_test) * 100 ))

In [12]:
overall_acc = overall_accuracy(y_test, y_pred)
m_prec = mean_prec(y_test, y_pred)
m_recall = mean_recall(y_test, y_pred)

print(f"Over all acc : {overall_acc*100:.2f}%\nmean prec : {m_prec*100:.2f}%\nmean recall : {m_recall*100:.2f}%")

Over all acc : 81.47%
mean prec : 79.16%
mean recall : 73.64%


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
